In [ ]:
# default_exp selection_parser


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#exporti
# Load Module's dependencies
from pathlib import Path
from pprint import pprint
from typing import *

import json
import pandas as pd
import fastcore.test as ft

In [ ]:
#exporti
import sc2reader

from sc_training.handle_tracker_event import *
from sc_training.macro_econ_parser import *

# Chapter 6 - Tracking Control Groups


In this chapter, I explore the data from the game's `ControlGroupEvents` and I define the module `controlgroup_parser` to compile this information as an indicator of the players' tactical performance. 

In [ ]:
#export

class CtrlGrpPlugin(object):
    """Tracks the composition of the Replay's Players Control Groups.

    Using this plug-in, the Replay object will include the `ctrl_grp_trk` 
    attribute. This attribute will store a dictionary of the control group 
    compositions using the replay's human players' ids (`pid`) as keys. 
    i.e.:
	
	`dict[pid (int):  control_group_compositions (dict)]`

    Each of these dictionaries uses the `second` attribute of a 
    `ControlGroupEvent` as an index and organises the control group 
    composition as one more dictionary, indexed with the integers 1 to 9, 
    which stores a list of units that compose each one of the nine control 
    groups of the player. i.e.

    `dict[pid(int): dict[second(int): dict[group(int): list_of_units]]]`
    """
    from sc2reader.engine.plugins import SelectionTracker  
    sc2reader.engine.register_plugin(SelectionTracker())
    name = "CtrlGrpPlugin"

    def handleInitGame(self, event, replay):
        replay.ctrl_grp_trk = dict()
        for human in replay.humans:
            selection = human.selection
            replay.ctrl_grp_trk[human.pid] = {0:{k: v for k, v 
                                                in selection.items()
                                                if 0<k<10}}

    def handleEvent(self, event, replay):

        if isinstance(event, sc2reader.events.game.ControlGroupEvent):
            selection = event.player.selection
            replay.ctrl_grp_trk[event.player.pid][event.second] = {
                                                        k: v for k, v 
                                                        in selection.items()
                                                        if 0<k<10} 

sc2reader.engine.register_plugin(CtrlGrpPlugin())

In [ ]:
show_doc(CtrlGrpPlugin, title_level=4)

<h4 id="CtrlGrpPlugin" class="doc_header"><code>class</code> <code>CtrlGrpPlugin</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>CtrlGrpPlugin</code>()

Tracks the composition of the Replay's Players Control Groups.

Using this plug-in, the Replay object will include the `ctrl_grp_trk` 
attribute. This attribute will store a dictionary of the control group 
compositions using the replay's human players' ids (`pid`) as keys. 
i.e.:
    
    `dict[pid (int):  control_group_compositions (dict)]`

Each of these dictionaries uses the `second` attribute of a 
`ControlGroupEvent` as an index and organises the control group 
composition as one more dictionary, indexed with the integers 1 to 9, 
which stores a list of units that compose each one of the nine control 
groups of the player. i.e.

`dict[pid(int): dict[second(int): dict[group(int): list_of_units]]]`

In [ ]:
#
# Load sample replays


RPS_PATH = Path("./test_replays")

game_path = str(RPS_PATH/"Jagannatha LE.SC2Replay")
single_replay = sc2reader.load_replay(game_path)
ctrl_grp_test = sc2reader.load_replay(str(RPS_PATH/'ctrl_grp_t.SC2Replay'))
ctrl_grp_test_2 = sc2reader.load_replay(str(RPS_PATH/'ctrl_grp_t_2.SC2Replay'))

ta_test = sc2reader.load_replay(str(RPS_PATH/'Terran_abilities.SC2Replay'))
pa_test = sc2reader.load_replay(str(RPS_PATH/'ProtossAbilities.SC2Replay'))

tms_test = sc2reader.load_replay(str(RPS_PATH/'TMovesSelect.SC2Replay'))
pms_test = sc2reader.load_replay(str(RPS_PATH/'p_move_test.SC2Replay'))

In [ ]:
print(ctrl_grp_test.length.seconds)
ctrl_grp_test.ctrl_grp_trk[1].keys()

535


dict_keys([0, 37, 49, 56, 59, 60, 67, 79, 100, 103, 116, 125, 142, 146, 160, 165, 176, 179, 190, 193, 194, 229, 233, 244, 249, 255, 276, 280, 303, 310, 328, 333, 387, 408, 415, 428, 444, 450, 455, 487, 497, 525, 550, 565, 584, 586, 608, 629, 640, 644, 657, 667, 699, 700, 718, 723, 726])

In [ ]:
TEST_MATCH = single_replay
TEST_PID = 1

ctrl_grp_e = [e for e in TEST_MATCH.events 
              if isinstance(e, sc2reader.events.game.ControlGroupEvent)
              and e.pid == TEST_PID]

set_ctrl_grp = [e for e in ctrl_grp_e
                if isinstance(e, sc2reader.events.game.SetControlGroupEvent)]

get_ctrl_grp = [e for e in ctrl_grp_e
                if isinstance(e, sc2reader.events.game.GetControlGroupEvent)]

add_ctrl_grp = [e for e in ctrl_grp_e
                if isinstance(e, 
                              sc2reader.events.game.AddToControlGroupEvent)]

print(f'ControlGroupEvents: {len(ctrl_grp_e)}')
print(f'SetControlGroupEvents: {len(set_ctrl_grp)}')
print(f'GetControlGroupEvents: {len(get_ctrl_grp)}')
print(f'AddControlGroupEvents: {len(add_ctrl_grp)}')


ControlGroupEvents: 100
SetControlGroupEvents: 14
GetControlGroupEvents: 86
AddControlGroupEvents: 0


In [ ]:
select_e = [e for e in TEST_MATCH.events 
              if isinstance(e, sc2reader.events.game.SelectionEvent)
              and e.pid == TEST_PID]

print(len(select_e))
print(set([s.control_group for s in select_e]))

215
{1, 3, 4, 5, 10}


In [ ]:
sel_e_times = set([s.second for s in select_e])
ctrl_grp_times = set([c.second for c in ctrl_grp_e])
print(f'select_e_times_indexes: {len(sel_e_times)}')
print(f'ctrl_grp_times_indexes: {len(ctrl_grp_times)}')

intersect = sel_e_times.intersection(ctrl_grp_times)
not_intersect_set = sel_e_times.symmetric_difference(ctrl_grp_times)
print(f'Number of elements that intersect {len(intersect)}')
print(f'Number of elements that do not intersect {len(not_intersect_set)}')
print(f'Sum of select and ctrl_group: {len(sel_e_times | ctrl_grp_times)}')

select_e_times_indexes: 179
ctrl_grp_times_indexes: 85
Number of elements that intersect 7
Number of elements that do not intersect 250
Sum of select and ctrl_group: 257


In [ ]:
ctrl_intersects = [c for c in ctrl_grp_e if c.second in intersect]
select_intersects = [s for s in select_e if s.second in intersect]

pprint(ctrl_intersects[0].__dict__)
pprint(select_intersects[0].__dict__)

{'bank': 1,
 'control_group': 1,
 'frame': 2625,
 'hotkey': 1,
 'is_local': True,
 'mask_data': None,
 'mask_type': 'None',
 'name': 'GetControlGroupEvent',
 'pid': 1,
 'player': Player 2 - MxChrisxM (Terran),
 'second': 164,
 'update_type': 2}
{'bank': 1,
 'control_group': 1,
 'frame': 2624,
 'is_local': True,
 'mask_data': [],
 'mask_type': 'ZeroIndices',
 'name': 'SelectionEvent',
 'new_unit_ids': [51380225],
 'new_unit_info': [(51380225, 155, 34, 1)],
 'new_unit_types': [(155, 34, 1, 1)],
 'new_units': [OrbitalCommand [3100001]],
 'objects': [OrbitalCommand [3100001]],
 'pid': 1,
 'player': Player 2 - MxChrisxM (Terran),
 'second': 164,
 'subgroup_index': 0}


TODO:
Define functions:

- `calc_ctrlg_ratio` to get the ratios of control group actions v commands for the different stages of the game. 
    - Should this ratio be calculated adding this commands to the action count?
    - Maybe is better just to get a count?
- `max_active_groups` get the maxium number of active control groups per game stage
- `calc_group_select_ratio` can I calculate a ratio of selecting with group vs selecting in total (group and rightclick)?

In [ ]:
def calc_ctrlg_ratio(rpl: sc2reader.resources.Replay, pid: int):
    pass